<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.


- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.

- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).

- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.


In [1]:
import pandas as pd

from src.support_db import establecer_conn, crear_db
import src.support_queries as sqc

### Carga de datos

In [2]:
df_demanda = pd.read_csv('datos/output/demanda.csv')
df_demo = pd.read_csv('datos/output/demografico.csv')
df_econ = pd.read_csv('datos/output/economico.csv')
df_generacion = pd.read_csv('datos/output/generacion.csv')
df_provincias = pd.read_csv('datos/output/provincias.csv')

Para relacionar adecuadamente las tablas necesitamos una tabla adicional. Usaremos `region_id`para establecer esta relación.

In [3]:
id_comunidades = {'Ceuta': 8744,
                'Melilla': 8745,
                'Andalucía': 4,
                'Aragón': 5,
                'Cantabria': 6,
                'Castilla - La Mancha': 7,
                'Castilla y León': 8,
                'Cataluña': 9,
                'País Vasco': 10,
                'Principado de Asturias': 11,
                'Comunidad de Madrid': 13,
                'Comunidad Foral de Navarra': 14,
                'Comunitat Valenciana': 15,
                'Extremadura': 16,
                'Galicia': 17,
                'Illes Balears': 8743,
                'Canarias': 8742,
                'Región de Murcia': 21,
                'La Rioja': 20}

# Nos vale con una lista de tuplas
valores_region = list(id_comunidades.items())

In [4]:
# Creamos la base de datos
crear_db('energia')

La base de datos ya existe


In [5]:
# Creacion tablas
# Establecemos conexión
conexion = establecer_conn("energia", "admin", "my_user")

# Creamos cada una de las tablas de la base de datos
conexion.cursor().execute(sqc.query_creacion_regiones)
conexion.cursor().execute(sqc.query_creacion_provincias)
conexion.cursor().execute(sqc.query_creacion_demografico)
conexion.cursor().execute(sqc.query_creacion_economico)
conexion.cursor().execute(sqc.query_creacion_generacion)
conexion.cursor().execute(sqc.query_creacion_demanda)
conexion.close()

In [6]:
valores_provincias = [tuple(row) for row in df_provincias.values]
valores_demo = [tuple(row) for row in df_demo.values]
valores_econ = [tuple(row) for row in df_econ.values]
valores_generacion = [tuple(row) for row in df_generacion.values]
valores_demanda = [tuple(row) for row in df_demanda.values]

In [7]:
# Inserción datos
# Establecemos conexión
conexion = establecer_conn("energia", "admin", "my_user")
cursor = conexion.cursor()

cursor.executemany(sqc.query_insercion_regiones, valores_region)
cursor.executemany(sqc.query_insercion_provincias, valores_provincias)
cursor.executemany(sqc.query_insercion_demografico, valores_demo)
cursor.executemany(sqc.query_insercion_economico, valores_econ)
cursor.executemany(sqc.query_insercion_generacion, valores_generacion)
cursor.executemany(sqc.query_insercion_demanda, valores_demanda)

cursor.close()
conexion.close()